# Formulate Helicity Amplitude Model for pγ→ηπ⁰p symbolically

## Generate Transitions

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
from ampform.io import aslatex, improve_latex_rendering
from IPython.display import Math
from qrules.particle import load_pdg

improve_latex_rendering()
particle_db = load_pdg()

In [ ]:
from qrules.particle import Particle, Spin

pgamma = Particle(
    name="pgamma",
    latex=r"p\gamma (s1/2)",
    spin=0.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99990,
)
pgamma

In [ ]:
pgamma2 = Particle(
    name="pgamma2",
    latex=R"p\gamma (s3/2)",
    spin=1.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99991,
)
pgamma2

In [ ]:
particle_db.add(pgamma)
particle_db.add(pgamma2)

In [ ]:
particle_db["N(1680)+"]

In [ ]:
particle_db["N(1650)+"]

In [ ]:
particle_db["N(1440)+"]

In [ ]:
particle_db["N(1520)+"]

In [ ]:
particle_db["N(1535)+"]

In [ ]:
particle_db["Delta(1232)+"]

In [ ]:
particle_db["a(2)(1320)0"]

In [ ]:
import graphviz
import qrules

reaction1_strong_EM = qrules.generate_transitions(
    initial_state=("pgamma"),
    final_state=["eta", "pi0", "p"],
    allowed_interaction_types=["strong", "EM"],
    formalism="canonical-helicity",
    particle_db=particle_db,
    max_angular_momentum=3,
    max_spin_magnitude=3,
    mass_conservation_factor=0,
)

In [ ]:
dot_se = qrules.io.asdot(reaction1_strong_EM, collapse_graphs=True)
graphviz.Source(dot_se)

From [PDG Live](https://pdglive.lbl.gov/):
For the record.

Branching fraction of $\Delta(1232)$ to 

| Mode     | Fraction |
| ----------- | ----------- |
| N$\pi$    | 99.4%         |
| N$\eta$   | 0.55-0.65%    |


Branching fraction of $N(1440)$ to 
| Mode     | Fraction |
| ----------- | ----------- |
| N$\pi$    | 55 - 75%      |
| N$\eta$   |    <1%        |

Branching fraction of $N(1520)$ to 
| Mode     | Fraction |
| ----------- | ----------- |
| N$\pi$    | 55 - 65%      |
| N$\eta$   | 0.07 - 0.09%  |

Branching fraction of $N(1535)$ to 
| Mode     | Fraction |
| ----------- | ----------- |
| N$\pi$    | 32-52%        |
| N$\eta$   | 30-55%        |

For simplicity, under the current `qrules` results, we limit the intermediate resonances to be:

"$a(2)$" for $\eta \pi^0$, 
"$\Delta(1232)$ for $\pi p$"
Both "$N(1535)$" for $\pi^0 p$
and $\eta p$

In [ ]:
import graphviz
import qrules

reaction1_strong_EM = qrules.generate_transitions(
    initial_state=("pgamma"),
    final_state=["eta", "pi0", "p"],
    allowed_intermediate_particles=["a(2)(1320)", "N(1535)", "Delta(1232)"],
    allowed_interaction_types=["strong", "EM"],
    formalism="canonical-helicity",
    particle_db=particle_db,
    max_angular_momentum=3,
    max_spin_magnitude=3,
    mass_conservation_factor=0,
)

In [ ]:
dot_se = qrules.io.asdot(reaction1_strong_EM, collapse_graphs=True)
graphviz.Source(dot_se)

## Build Helicity Amplitude Models

we use ampform to formulate the transitions as an amplitude model (here: HelicityModel). 

We choose to use relativistic_breit_wigner_with_ff() as the lineshape for all resonances and use a Blatt-Weisskopf form factor (create_non_dynamic_with_ff()) for the production decay. 

In [ ]:
import ampform
from ampform.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner,
)

model_builder = ampform.get_builder(reaction1_strong_EM)
# model_builder.dynamics.assign("pgamma", create_non_dynamic_with_ff)
model_builder.config.scalar_initial_state_mass = True
model_builder.config.stable_final_state_ids = 0, 1, 2
for name in reaction1_strong_EM.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, create_relativistic_breit_wigner)
model = model_builder.formulate()

In [ ]:
_, first_item, *_rest = model.amplitudes.items()
symbol, expr = first_item

In [ ]:
from ampform.io import aslatex

(symbol, expr), *_ = model.amplitudes.items()
Math(aslatex({symbol: expr}, terms_per_line=1))

The first component of the full amplitude is shown above.

See below for all individual amplitudes for specific helicity states in the full amplitude.

In [ ]:
Math(aslatex(model.amplitudes))

In [ ]:
type(model.intensity)

### Parameters

The parameters of the helicity amplitude model::

In [ ]:
sorted_parameter_defaults = {
    symbol: model.parameter_defaults[symbol]
    for symbol in sorted(model.parameter_defaults, key=str)
}

src = aslatex(sorted_parameter_defaults)
Math(src)

Optionally, we can backup the HelicityModel to disk via pickle.

In [ ]:
import pickle

qrules.io.write(reaction1_strong_EM, "transitions.json")
with open("helicity_model.pickle", "wb") as stream:
    pickle.dump(model, stream)

In [ ]:
from ampform.helicity import HelicityModel

with open("helicity_model.pickle", "rb") as model_file:
    imported_model: HelicityModel = pickle.load(model_file)

### Reaction info

In [ ]:
initial_state, *_ = imported_model.reaction_info.initial_state.values()
print("Initial state:")
print("-1: ", initial_state.name)
print("Final state:")
for i, p in imported_model.reaction_info.final_state.items():
    print(f"  {i}: {p.name}")
del initial_state

## Generate phase space sample

In [ ]:
phsp_event = 1_000_000

In [ ]:
from tensorwaves.data import TFPhaseSpaceGenerator, TFUniformRealNumberGenerator

rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction1_strong_EM.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction1_strong_EM.final_state.items()},
)
phsp_momenta = phsp_generator.generate(phsp_event, rng)

In [ ]:
import numpy as np
import pandas as pd

pd.DataFrame(
    {
        (k, label): np.transpose(v)[i]
        for k, v in phsp_momenta.items()
        for i, label in enumerate(["E", "px", "py", "pz"])
    }
)

In [ ]:
import numpy as np

p = np.array(list(phsp_momenta.values()))
p.shape

In [ ]:
p.sum(axis=0).round(decimals=14)

In [ ]:
E0 = p[0].T[0]
px0 = p[0].T[1]
py0 = p[0].T[2]
pz0 = p[0].T[3]
p0 = np.sqrt(px0**2 + py0**2 + pz0**2)

E1 = p[1].T[0]
px1 = p[1].T[1]
py1 = p[1].T[2]
pz1 = p[1].T[3]
p1 = np.sqrt(px1**2 + py1**2 + pz1**2)

E2 = p[2].T[0]
px2 = p[2].T[1]
py2 = p[2].T[2]
pz2 = p[2].T[3]
p2 = np.sqrt(px2**2 + py2**2 + pz2**2)

## Visualize kinematic variables

In [ ]:
from tensorwaves.function.sympy import create_parametrized_function

unfolded_expression = model.expression.doit()
intensity_func = create_parametrized_function(
    expression=unfolded_expression,
    parameters=model.parameter_defaults,
    backend="numpy",
)

In [ ]:
Math(aslatex(model.kinematic_variables))

In [ ]:
from tensorwaves.data import SympyDataTransformer

helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="jax"
)

In [ ]:
phsp = helicity_transformer(phsp_momenta)
list(phsp)

In [ ]:
for state_id, particle in reaction1_strong_EM.final_state.items():
    print(f"ID {state_id}:", particle.name)

In [ ]:
import pandas as pd

phsp_frame = pd.DataFrame(phsp)
phsp_frame

## Visualize model and  intensity-based sample

In [ ]:
%config InlineBackend.figure_formats = ['png']
fig, ax = plt.subplots()
hist = ax.hist2d(
    phsp["m_01"].real ** 2,
    phsp["m_12"].real ** 2,
    bins=200,
    cmin=1e-6,
    density=True,
    cmap="jet",
    # vmax=0.15,
)
ax.set_title("Phase Space Dalitz Plot")
ax.set_xlabel(R"$m^2(\eta \pi^0)\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$m^2(\pi^0 p)\;\left[\mathrm{GeV}^2\right]$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
# %config InlineBackend.figure_formats = ['png']

fig, ax = plt.subplots()
hist = ax.hist2d(
    phsp["m_01"].real ** 2,
    phsp["m_12"].real ** 2,
    bins=200,
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
    weights=intensity_func(phsp),
)
ax.set_title("Model-weighted Phase space Dalitz Plot")
ax.set_xlabel(R"$m^2(\eta \pi^0)\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$m^2(\pi^0 p)\;\left[\mathrm{GeV}^2\right]$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt

resonances = sorted(
    reaction1_strong_EM.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(
    np.real(phsp_frame["m_01"]),
    bins=100,
    alpha=0.5,
    density=True,
    # color = "orange",
    label="Weighted PHSP",
    weights=intensity_func(phsp),
)
# ax.hist(
#     np.real(data_frame["m_12"]),
#     bins=100,
#     alpha=0.5,
#     density=True,
#     color = colors[0],
# )
# ax.hist(
#     np.real(data_frame["m_02"]),
#     bins=100,
#     alpha=0.5,
#     density=True,
#     color = colors[2],
# )
ax.set_xlabel(R"$m(\eta \pi^0)$ [GeV]")
# for p, color in zip(resonances, colors):
#     ax.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
ax.axvline(
    x=resonances[1].mass, linestyle="dotted", label=resonances[1].name, color="black"
)
ax.legend()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt

resonances = sorted(
    reaction1_strong_EM.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(
    np.real(phsp_frame["m_12"]),
    bins=100,
    alpha=0.5,
    density=True,
    # color = colors[0],
    label="Weighted PHSP",
    weights=intensity_func(phsp),
)

ax.set_xlabel(R"$m(\pi^0 p)$ [GeV]")
# for p, color in zip(resonances, colors):
#     ax.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
ax.axvline(
    x=resonances[0].mass, linestyle="dotted", label=resonances[0].name, color=colors[0]
)
ax.axvline(
    x=resonances[2].mass, linestyle="dotted", label=resonances[2].name, color=colors[2]
)
ax.legend()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt

resonances = sorted(
    reaction1_strong_EM.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(
    np.real(phsp_frame["m_02"]),
    bins=100,
    alpha=0.5,
    density=True,
    # color = colors[2],
    label="Weighted PHSP",
    weights=intensity_func(phsp),
)
ax.set_xlabel(R"$m(\eta p)$ [GeV]")
# for p, color in zip(resonances, colors):
#     ax.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
ax.axvline(
    x=resonances[2].mass, linestyle="dotted", label=resonances[2].name, color=colors[2]
)
ax.legend()
plt.show()